In [ ]:
#########################################

# ENTER MODEL WHICH YOU RUN

#########################################

model = 'glm_complex_anova'

In [ ]:
"""
###############
1. the function of the volume rewrite was incorrect
2. there was an error in dummy.fsf, the output file should be not "run-01",
   but "run-01.feat"
###############
"""

import re
import shutil
import nibabel as nib
import os
import csv
import fileinput
import tempfile
import nipype.interfaces.fsl as fsl

In [ ]:
def make_fsf_run(sub, run):
    new = [line.replace('sub-01', sub) for line in dummy]
    new = [line.replace('run-01', run) for line in new]
    with open(os.path.join(DATA_DIR, sub, 'feat-design_'+run+'.fsf'), 'w') as fname:
        for line in new:
            fname.write(line)
            
def rewrite_volumes_num(sub,run):
    fsf_file_path = os.path.join(DATA_DIR, sub, 'feat-design_'+run+'.fsf')
    NIFTI_DIR = pth+"/preprocess/"+sub+"/func"

    nifti_file_path = NIFTI_DIR+"/"+sub+"_task-reproduction_"+run+"_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"

    nifti_file = nib.load(nifti_file_path)
    actual_num_volumes = nifti_file.shape[3]
    print(f'number of volumes {actual_num_volumes}')
    # a line that starts with "set fmri(npts)" + one or more digits (represented by "\d+"
    # r before the quotes tell that "\" should be left as it is
    find_regex = r"(set fmri\(npts\)) \d+"
    replace_regex = r"\g<1> {}".format(actual_num_volumes)
    
    # Open the FSF file and read its contents
    with open(fsf_file_path, "r") as fsf_file:
        fsf_contents = fsf_file.read()
        
    # Use regular expressions to find and replace the number of volumes
    fsf_contents = re.sub(find_regex, replace_regex, fsf_contents,flags=re.MULTILINE)
    
    with open(fsf_file_path, "w") as fsf_file:
        fsf_file.write(fsf_contents)

In [ ]:
pth = '/home/lisz/Desktop/Time_project/time_fmri/data/derivatives'

DATA_DIR = pth+'/'+model+'/glm_1st_level'

runs = ["run-01","run-02","run-03","run-04"]

pth_sub_list = "/home/lisz/Desktop/Time_project/time_fmri/data"

with open(pth_sub_list+'/participants.tsv', 'r') as tsv_file:
    subjects_list = ['sub-{0:02d}'.format(int(row['participant_id'])) for row in csv.DictReader(tsv_file, delimiter='\t')]

In [ ]:
for sub in subjects_list:
    print(f'Processing {sub}')
    for run in runs:
        print(f'Processing      {run}')
        if not os.path.isdir(os.path.join(DATA_DIR, sub)):
            os.mkdir(os.path.join(DATA_DIR, sub))
        with open(os.path.join(DATA_DIR, 'draft.fsf'), 'r') as fname:
            dummy = fname.readlines()
            make_fsf_run(sub, run)
            rewrite_volumes_num(sub,run)